# `hyprovo` Tests

## Imports

In [1]:
(defmacro eval-file [^(of str) filename]
  "Function to evaluate Hy expressions in `filename`. Returns count of expressions
   evaluated. "
  `(with [f (open ~filename)]
     (setv expr-count 0)
     (try (while True
            (setv expr (read f))
            (setv expr-count (inc expr-count))
            (eval expr))
          (except [e EOFError]
            (print expr-count)))))

<function eval_file at 0x7f99000d8950>

In [2]:
(require [hyprovo.framework [setup-test-env 
                                 teardown-test-env 
                                 report-result
                                 check
                                 report-combined-results
                                 deffixture
                                 with-fixture
                                 with-fixtures
                                 deftest
                                 defsuite]])

(import [hyprovo.framework [combine-results run-tests ▶]])

(import [hyprovo.logger [null-logger 
                             console-logger
                             file-logger 
                             console-and-file-logger]])

(require [hy.extra.anaphoric [*]])

(import os)
(setv make-path (. os path join))

## Global Variables

In [3]:
(setv *test-plan* [])

In [4]:
(import [datetime [datetime]])

In [5]:
(defn now-str []
      "Return current timestamp as string upto precision of seconds."
      (.strftime (.now datetime) "%Y%m%d%H%M%S"))

;; (now-str)
(setv *hyprovo-logger* (console-and-file-logger :log-file (make-path "logs" (+ (now-str) ".log"))))

File does not exist


## Macro and Function Definitions

In [7]:
(defmacro log-actual-result [test-name &rest body]
  `(defn ~test-name [&optional [test-logger console-logger]]
     (setup-test-env)
     (setv *test-logger* test-logger)
     (setv results None)
     ~@body
     (return results)))

<function log_actual_result at 0x7f98f296e0d0>

In [8]:
(defn contains-timestamp? [^(of str) line]
    (-> (map #%(in %1 line) ["INFO" "ERROR" "DEBUG"]) (any)))

(defn read-log [^(of str) log-file &optional ^(of bool) [strip-timestamps True]]
    (with [file (open log-file)]
          (setv result% (.readlines file)))
      
    (setv logs [])
    (for [r% result%]
        (if (and strip-timestamps (contains-timestamp? r%))             
            (.append logs (cut r% 26))
            (.append logs r%)))
      
    (return logs))

In [9]:
; (read-log "tests/actual-report-result-pass.txt")

In [10]:
(defmacro! report [label description expr]
    `(do
      (if ~expr
          (do 
           (setv ~g!report% (, 1 1 0)) ; total, pass, fail
           (->> (.format "✓ Pass: {} - {}" ~label ~description) (.debug *hyprovo-logger*)))
          (do
           (setv ~g!report% (, 1 0 1)) ; total, pass, fail
           (->> (.format "✗ Fail: {} - {}" ~label ~description) (.error *hyprovo-logger*))))
      ~g!report%))

<function report at 0x7f98f29866a8>

In [11]:
(defmacro! validate-logs [label description]
    `(do
      (setv ~g!actual% (make-path "results" "actual" (.format "{}.txt" ~label))
            ~g!expected% (make-path "results" "expected" (.format "{}.txt" ~label))
            ~g!test% (= (read-log ~g!actual%) (read-log ~g!expected%)))
      (report ~label ~description ~g!test%)))

<function validate_logs at 0x7f98f2986950>

In [12]:
(defn sum-results [results]
    (as-> (zip #* results) it
          (map sum it)
          (tuple it)))

In [13]:
(defn compare-log-and-return-value [label actual expected description]
  (setv log-file% (make-path "results" "actual" (.format "{}.txt" label)))
  
  ;; compare log output
  (setv actual% (actual :test-logger (file-logger :log-file log-file%)))
  (setv result-log% (validate-logs label description))

  ;; compare return value
  (setv results% (= expected actual%))
  (setv result-return% (report label "Return value" results%))

  ;; combine results
  (sum-results (, result-log% result-return%)))

In [14]:
(defmacro generate-test-case/result= [label actual expected description]
   `(fn []
          (log-actual-result test% (setv results ~actual))          
          (compare-log-and-return-value ~label test% ~expected ~description)))

<function hyx_generate_test_caseXsolidusXresultXequals_signX at 0x7f98f292ad08>

In [15]:
(defmacro add-to-test-plan/result= [label actual expected description]
          `(.append *test-plan* (generate-test-case/result= ~label ~actual ~expected ~description)))

<function hyx_add_to_test_planXsolidusXresultXequals_signX at 0x7f98f292ae18>

## `report-results`

In [16]:
(add-to-test-plan/result= "report-result-pass" 
                          (report-result = (+ 1 2) 3 "Passing result") 
                          True 
                          "Passing output format")

In [17]:
(add-to-test-plan/result= "report-result-fail"
                          (report-result = (+ 1 2) 5 "Failing result")
                          False
                          "Failing output format")

## `check`

In [18]:
(setv test-case-check '(check
                          (= (+ 1 2) 3 "add pass")
                          (= (- 2 3) 4 "sub fail")
                          (= (/ 10 2) 3 "div fail")))

In [19]:
(add-to-test-plan/result= "check" 
                          (eval test-case-check)
                          (, 3 1 2)
                          "Multiple passing and failing statements")

## Combining Results

### `combine-results`

In [20]:
(setv test-case-combine-results '(combine-results
                     (check
                      (= (+ 1 2) 3 "add pass")
                      (= (- 2 3) 4 "sub fail")
                      (= (* 7 2) 14 "mul pass"))
                     (check
                      (= (/ 4 2) 2 "div pass")
                      (= (+ 2 3) -6 "add fail")
                      (= (* 10 2) 3 "mul fail"))))

In [21]:
(add-to-test-plan/result= "combine-results" 
                          (eval test-case-combine-results)
                          (, 6 3 3)
                          "Combine results across multiple `check` statements.")

### `report-combined-results`

In [22]:
(add-to-test-plan/result= "report-combined-results"
                          (-> (eval test-case-combine-results) (report-combined-results))
                          (, 6 3 3)
                          "Combine and report results across multiple `check` statements")

## Test Definitions

In [23]:
(deftest test-add-report-combined-results
  (-> (check
    (= (+ 1 2) 3 "add pass")
    (= (+ 2 3) 4 "add fail"))
      (combine-results)
      (report-combined-results)))

 (deftest test-add
  (check
    (= (+ 1 2) 3 "add pass")
    (= (+ 2 3) 4 "add fail")))

 (deftest test-add-variable
  (check
    (= (+ a b) 2 "add pass")
    (= (+ a b) 3 "add fail")))
 
 (deftest test-add-variable-local-defs
  (setv a 5 b 10)
  (check
    (= (+ a b) 15 "add pass")
    (= (+ a b) 3 "add fail")))
 
 (deftest test-add-variable-partial-local-defs
  (setv c 1)
  (check
    (= (+ a b c) 3 "add pass")
    (= (+ a b c) 3 "add fail")))

 (deftest test-sub
  (check
    (= (- 2 2) 0 "sub pass")
    (= (- 2 3) 4 "sub fail")))

 (deftest test-sub-variable
  (check
    (= (- a a) 0 "sub pass")
    (= (- a b) 4 "sub fail")))
 
(deftest test-sub-variable-local-defs
  (setv a 8 b 10)
  (check
    (= (- a b) -2 "sub pass")
    (= (- a b) 7 "sub fail")))
 
 (deftest test-sub-variable-partial-local-defs
  (setv c 3)
  (check
    (= (- a b c) -3 "add pass")
    (= (- a b c) -19 "add fail")))

(deftest test-mul
 (check
   (= (* 1 2) 2 "mul pass")
   (= (* 2 3) 4 "mul fail")))

(deftest test-mul-variable
 (check
   (= (* a b) 1 "mul pass")
   (= (* a b) 4 "mul fail")))

(deftest test-mul-variable-local-defs
  (setv a 2 b 7)
  (check
    (= (* a b) 14 "mul pass")
    (= (* a b) -5 "mul fail")))
 
 (deftest test-mul-variable-partial-local-defs
  (setv c 2)
  (check
    (= (* a b c) 2 "mul pass")
    (= (* a b c) -3 "mul fail")))

(deftest test-div
  (check
    (= (/ 2 1) 2 "div pass")
    (= (/ 2 3) 4 "div fail")))

(deftest test-div-variable
  (check
    (= (/ b a) 1 "div pass")
    (= (/ a b) 4 "div fail")))

 (deftest test-div-variable-local-defs
  (setv a 3 b 6)
  (check
    (= (/ b a) 2 "div pass")
    (= (/ a b) 3 "div fail")))
 
 (deftest test-div-variable-partial-local-defs
  (setv c 1)
  (check
    (= (/ b a c) 1.0 "div pass")
    (= (/ a b c) 11.3 "div fail")))

(deftest test-arithmetic-1-report-combined-results
        (-> (combine-results 
             (▶ (test-sub) (test-div))
               (check
                    (= (+ 1 1) 2 "add pass")
                    (= (- 0 1) 2 "sub fail")))
            (report-combined-results)))

(deftest test-arithmetic-2-report-combined-results
        (-> (▶ (test-add) (test-mul))
            (report-combined-results)))

(deftest test-arithmetic-variable-local-defs
        (-> (▶ (test-sub-variable-local-defs) 
               (test-mul-variable-local-defs))
            (report-combined-results)))

<function test_arithmetic_variable_local_defs at 0x7f98f2971ea0>

## `deftest`

In [24]:
(add-to-test-plan/result= "deftest" 
                          (▶ (test-add-report-combined-results))
                          (, 2 1 1)
                          "Report combined results in `check`")

In [25]:
(add-to-test-plan/result= "deftest-nested-and-check" 
                          (▶ (test-arithmetic-1-report-combined-results))
                          (, 6 3 3)
                          "Report combined results with nested tests and checks.")

In [26]:
(add-to-test-plan/result= "deftest-variable-local-defs" 
                          (▶ (test-sub-variable-local-defs))
                          (, 2 1 1)
                          "Variables defined locally in test body.")

## Fixtures

### Defintions (`deffixture`)

In [27]:
(deffixture fix-a ((setv a 1 b 1)) ((del a b)))
(deffixture fix-b ((setv a 2 b 3)) ((del a b)))
(deffixture fix-c ((setv a 1)) (None))
(deffixture fix-d ((setv b 1)) (None))

<function fix_d at 0x7f99000d8840>

### `with-fixture`

In [28]:
(add-to-test-plan/result= "with-fixture" 
                          (▶ (with-fixture fix-a (▶ (test-add-variable) (test-sub-variable))))
                          (, 4 2 2)
                          "Multiple tests with one fixture.")

### Nested `with-fixture`

In [29]:
(add-to-test-plan/result= "nested-with-fixture" 
                          (▶ (with-fixture fix-c 
                             (▶ (with-fixture fix-d 
                                (▶ (test-add-variable) (test-mul-variable))))))
                          (, 4 2 2)
                          "Multiple tests with nested fixtures.")

### `with-fixtures`

In [30]:
(add-to-test-plan/result= "with-fixtures" 
                          (▶ (with-fixtures [fix-a fix-b] (▶ (test-mul-variable) (test-div-variable))))
                          (, 8 2 6)
                          "Multiple tests using multiple fixtures.")

### Nested `with-fixtures`

In [31]:
(add-to-test-plan/result= "nested-with-fixtures" 
                          (▶ (with-fixtures [fix-c fix-c] 
                            (▶ (with-fixtures [fix-d fix-d] 
                              (▶ (test-add-variable) (test-sub-variable))))))
                          (, 16 8 8)
                          "Multiple tests using multiple nested fixtures.")

### `with-fixture` in `deftest`

In [57]:
(deftest deftest-with-fixture 
         (-> (combine-results
           (▶ (with-fixture fix-a (▶ (test-add-variable) (test-sub-variable))))
           (▶ (test-mul) (test-div)))
             (report-combined-results)))

<function deftest_with_fixture at 0x7f990012a510>

In [33]:
(add-to-test-plan/result= "deftest-with-fixture" 
                          (▶ (deftest-with-fixture))
                          (, 8 4 4)
                          "deftest with fixture and tests.")

### `deftest` with local variable defintions

In [34]:
(deftest deftest-local-defs-with-fixture 
         (-> (combine-results
           (▶ (with-fixture fix-a 
                            (▶ (test-add-variable-local-defs)
                                (test-div-variable)))))                               
             (report-combined-results)))

<function deftest_local_defs_with_fixture at 0x7f98f28b6bf8>

In [35]:
(add-to-test-plan/result= "deftest-local-defs-with-fixture" 
                          (▶ (deftest-local-defs-with-fixture))
                          (, 4 2 2)
                          "Values of local variables when defined within deftest should be used instead of those in the fixture.")

### `deftest` with partial local variable definitions

In [36]:
(deftest deftest-partial-local-defs-with-fixture 
         (-> (combine-results
           (▶ (with-fixture fix-a 
                            (▶ (test-sub-variable-partial-local-defs)
                                (test-mul-variable-partial-local-defs)))))                               
             (report-combined-results)))

<function deftest_partial_local_defs_with_fixture at 0x7f98f292a7b8>

In [37]:
(add-to-test-plan/result= "deftest-partial-local-defs-with-fixture" 
                          (▶ (deftest-partial-local-defs-with-fixture))
                          (, 4 2 2)
                          "Values of local variables defined within deftest should be used. Variable definitions missing should be taken from those defined in the fixture.")

## Suites

### Function Definitions

In [38]:
(defmacro generate-test-case/suite= [label actual expected description]
   `(fn []
          (compare-log-and-return-value ~label ~actual ~expected ~description)))

<function hyx_generate_test_caseXsolidusXsuiteXequals_signX at 0x7f98f2986d90>

In [39]:
(defmacro add-to-test-plan/suite= [label actual expected description]
          `(.append *test-plan* (generate-test-case/suite= ~label ~actual ~expected ~description)))

<function hyx_add_to_test_planXsolidusXsuiteXequals_signX at 0x7f98f2986730>

### With `check`

In [40]:
(defsuite suite-check
  (setv results (-> (combine-results 
       (check
           (= (+ 1 2) 3 "add pass")
           (= (- 2 3) 4 "sub fail")
           (= (/ 10 2) 3 "div fail")))
  (report-combined-results))))

In [41]:
(add-to-test-plan/suite= "suite-check" 
                         suite-check
                         (, 3 1 2)
                         "Report combined results in suite with checks.")

### With `deftest`

In [42]:
(defsuite suite-deftest
  (setv results (-> (combine-results 
       (▶ (test-add) (test-sub)))
  (report-combined-results))))

;; (suite-deftest)

In [43]:
(add-to-test-plan/suite= "suite-deftest" 
                         suite-deftest
                         (, 4 2 2)
                         "Report combined results in suite with deftests.")

### With nested `deftests`

In [44]:
(defsuite suite-nested-deftests
  (setv results (-> (combine-results 
       (▶ (test-arithmetic-1-report-combined-results) 
          (test-arithmetic-2-report-combined-results)))
  (report-combined-results))))

;; (suite-nested-deftests)

In [45]:
(add-to-test-plan/suite= "suite-nested-deftests" 
                         suite-nested-deftests
                         (, 10 5 5)
                         "Report combined results in suite with nested deftests.")

### Nested suites

In [46]:
(defsuite nested-suites
  (setv results 
        (-> (combine-results 
             (suite-check :test-logger test-logger)
             (suite-deftest :test-logger test-logger) 
             (suite-nested-deftests :test-logger test-logger))
            (report-combined-results))))
  
;; (nested-suites)

In [47]:
(add-to-test-plan/suite= "nested-suites" 
                         nested-suites
                         (, 17 8 9)
                         "Report combined results in suite with nested suites.")

### Suites `with-fixture` in `deftest`

In [48]:
(defsuite suite-with-fixture-in-deftest
          (setv results 
                (-> (combine-results
                     (▶ (deftest-with-fixture))))))

;; (suite-with-fixture-in-deftest)

In [49]:
(add-to-test-plan/suite= "suite-with-fixture-in-deftest" 
                         suite-with-fixture-in-deftest
                         (, 8 4 4)
                         "deftest in suite containing a fixture.")

### Suites `with-fixtures`

In [58]:
(defsuite suite-with-fixtures
          (setv results 
                (-> (combine-results
                     (▶ (with-fixtures [fix-a fix-b] 
                                       (▶ (test-mul-variable) 
                                          (test-div-variable))))))))

;; (suite-with-fixtures)

In [51]:
(add-to-test-plan/suite= "suite-with-fixtures" 
                         suite-with-fixtures
                         (, 8 2 6)
                         "Suite with multiple fixtures for multiple tests.")

### Nested suites with fixtures

In [52]:
(defsuite suite-with-nested-fixture
          (setv results 
                 (▶ (with-fixture fix-c 
                  (▶ (with-fixture fix-d (▶ (test-add-variable) 
                                             (test-mul-variable))))))))

;; (suite-with-nested-fixture)

In [53]:
(add-to-test-plan/suite= "suite-with-nested-fixture" 
                         suite-with-nested-fixture
                         (, 4 2 2)
                         "Suite with nested fixtures running multiple tests.")

### Nested suites with nested fixtures

In [54]:
(defsuite nested-suites-with-nested-fixtures
          (setv results 
                (-> (combine-results 
                     (suite-with-fixtures :test-logger test-logger)
                     (suite-with-nested-fixture :test-logger test-logger))
                    (report-combined-results))))

;; (nested-suites-with-nested-fixtures)

In [55]:
(add-to-test-plan/suite= "nested-suites-with-nested-fixtures" 
                         nested-suites-with-nested-fixtures
                         (, 12 4 8)
                         "Suite with nested suites and nested fixtures.")

## Run Tests!

In [56]:
(as-> (sum-results (map #%(%1) (distinct *test-plan*))) it
      (if (= (get it 2) 0)
          (->> (.format "All tests PASSED {}" (repr it)) (.info *hyprovo-logger*))
          (->> (.format "{} tests FAILED {}" (get it 2) (repr it)) (.info *hyprovo-logger*))))

2021-11-11 10:37:39.267 - DEBUG - ✓ Pass: report-result-pass - Passing output format
2021-11-11 10:37:39.269 - DEBUG - ✓ Pass: report-result-pass - Return value
2021-11-11 10:37:39.302 - DEBUG - ✓ Pass: report-result-fail - Failing output format
2021-11-11 10:37:39.303 - DEBUG - ✓ Pass: report-result-fail - Return value
2021-11-11 10:37:39.416 - DEBUG - ✓ Pass: check - Multiple passing and failing statements
2021-11-11 10:37:39.417 - DEBUG - ✓ Pass: check - Return value
2021-11-11 10:37:39.610 - DEBUG - ✓ Pass: combine-results - Combine results across multiple `check` statements.
2021-11-11 10:37:39.612 - DEBUG - ✓ Pass: combine-results - Return value
2021-11-11 10:37:39.797 - DEBUG - ✓ Pass: report-combined-results - Combine and report results across multiple `check` statements
2021-11-11 10:37:39.799 - DEBUG - ✓ Pass: report-combined-results - Return value
2021-11-11 10:37:39.869 - DEBUG - ✓ Pass: deftest - Report combined results in `check`
2021-11-11 10:37:39.870 - DEBUG - ✓ Pass: 